# Benchmarking skforecast

This notebook benchmarks the performance (velocity) of the `skforecast` in its different versions and keeps track of the results.

**Notes**

+ In version `0.15.0` the binning of residuals was introduced in multi-series forecasters. This explains the increase in the time taken to fit the model.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

/home/joaquin/Documents/GitHub/skforecast


In [2]:
# !pip install --upgrade numpy < 2.0 && \
# pip install --upgrade pandas && \
# pip install --upgrade scikit-learn

In [3]:
# !pip install skforecast==0.15.1
# !pip install skforecast==0.14.0
# !pip install skforecast==0.13.0

In [4]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import sklearn
import joblib
from benchmarking import BenchmarkRunner, plot_benchmark_results
import skforecast
import platform
import psutil
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

from skforecast.utils import *
if skforecast.__version__ >= '0.14.0':
    from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
    from skforecast.direct import ForecasterDirect, ForecasterDirectMultiVariate
    from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster, backtesting_forecaster_multiseries
else:
    from skforecast.ForecasterAutoreg import ForecasterAutoreg
    from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
    from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
    from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
    from skforecast.model_selection import backtesting_forecaster, backtesting_forecaster_multiseries

In [5]:
print(f"Python version: {platform.python_version()}")
print(f"skforecast version: {skforecast.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"Computer network name: {platform.node()}")
print(f"Processor type: {platform.processor()}")
print(f"Platform type: {platform.platform()}")
print(f"Operating system: {platform.system()}")
print(f"Operating system release: {platform.release()}")
print(f"Operating system version: {platform.version()}")
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.12.9
skforecast version: 0.16.0
scikit-learn version: 1.6.1
pandas version: 2.2.3
numpy version: 2.2.5
Computer network name: joaquin-HP-ProBook-440-G6
Processor type: x86_64
Platform type: Linux-6.11.0-24-generic-x86_64-with-glibc2.39
Operating system: Linux
Operating system release: 6.11.0-24-generic
Operating system version: #24~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Mar 25 20:14:34 UTC 2
Number of physical cores: 4
Number of logical cores: 8


In [6]:
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="'force_all_finite' was renamed to 'ensure_all_finite'"
)

# ForecasterRecursiveMultiSeries

In [7]:
# Mock data for benchmarking
# ==========================================================
n_series = 600
len_series = 2000
series_dict = {}
rng = np.random.default_rng(321)
for i in range(n_series):
    series_dict[f'series_{i}'] = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name=f'series_{i}'
    )
series_dataframe = pd.DataFrame(series_dict, index=series_dict['series_0'].index)

exog_dict = {}
rng = np.random.default_rng(321)
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=series_dict[k].index
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict[k] = exog

exog_dict_prediction = {}
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=pd.date_range(
                start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                periods=100,
                freq='h'
            )
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict_prediction[k] = exog

In [8]:
# Benchmarking speed ForecasterRecursiveMultiSeries
# ==============================================================================
regressor = LGBMRegressor(random_state=8520, verbose=-1)
# regressor = LinearRegression()
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursiveMultiSeries(
        regressor=regressor,
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )
else:
    forecaster = ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )

def ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict():
    forecaster.fit(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict():
    forecaster._create_train_X_y(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog():
    forecaster.fit(series=series_dataframe)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog():
    forecaster._create_train_X_y(series=series_dataframe)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_exog_dict():
    forecaster._create_train_X_y(series=series_dataframe, exog=exog_dict)

def ForecasterRecursiveMultiSeries__create_train_X_y_single_series():
    _ = forecaster._create_train_X_y_single_series(
            y = series_dict['series_0'],
            exog = exog_dict['series_0'],
            ignore_exog = False,
        )

def ForecasterRecursiveMultiSeries_predict_exog_is_dict():
    forecaster.predict(steps=100, exog=exog_dict_prediction, suppress_warnings=True)

def ForecasterRecursiveMultiSeries_predict_interval_exog_is_dict_conformal():
    forecaster.predict_interval(
        steps=100,
        exog=exog_dict_prediction,
        method='conformal',
        interval=[5, 95],
        suppress_warnings=True
    )

def ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict():
    _ = forecaster._create_predict_inputs(
            steps         = 100,
            exog         = exog_dict_prediction,
            check_inputs = True
        )

def ForecasterRecursiveMultiSeries__check_predict_inputs():
    check_predict_input(
        forecaster_name  = type(forecaster).__name__,
        steps            = 100,
        is_fitted        = forecaster.is_fitted,
        exog_in_         = forecaster.exog_in_,
        index_type_      = forecaster.index_type_,
        index_freq_      = forecaster.index_freq_,
        window_size      = forecaster.window_size,
        last_window      = pd.DataFrame(forecaster.last_window_),
        exog             = exog_dict_prediction,
        exog_type_in_    = forecaster.exog_type_in_,
        exog_names_in_   = forecaster.exog_names_in_,
        interval         = None,
        levels           = forecaster.series_names_in_,
        series_names_in_ = forecaster.series_names_in_,
        encoding         = forecaster.encoding
    )

def ForecasterRecursiveMultiSeries_backtesting_series_is_dict_exog_is_dict():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dict,
                exog=exog_dict,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dict,
                exog=exog_dict,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )
        
def ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_no_exog():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )
        
def ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )
        
def ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict_conformal():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                cv=cv,
                interval=[5, 95],
                interval_method='conformal',
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                interval=[5, 95],
                interval_method='conformal',
                metric='mean_squared_error',
                show_progress=False
            )


runner = BenchmarkRunner(repeat=10, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_exog_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_single_series, forecaster=forecaster)

runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog, forecaster=forecaster)

forecaster.fit(series=series_dict, exog=exog_dict)
runner = BenchmarkRunner(repeat=10, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_predict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_predict_interval_exog_is_dict_conformal, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__check_predict_inputs, forecaster=forecaster)

runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_no_exog, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict_conformal, forecaster=forecaster)

Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_exog_dict with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_single_series with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries_predict_exog_is_dict with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries_predict_interval_exog_is_dict_conformal with skforecast version 0.16.0


/home/joaquin/Documents/GitHub/skforecast/benchmarks/benchmarking.py:63: UserWarning: The function ForecasterRecursiveMultiSeries_predict_interval_exog_is_dict_conformal raised an exception: ForecasterRecursiveMultiSeries.predict_interval() got an unexpected keyword argument 'nterval'
  warnings.warn(f"The function {func.__name__} raised an exception: {e}")


Benchmarking function: ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries__check_predict_inputs with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries_backtesting_series_is_dict_exog_is_dict with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_no_exog with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict with skforecast version 0.16.0
Benchmarking function: ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict_conformal with skforecast version 0.16.0


In [9]:
# Plot results
# ==============================================================================
display_df = True
selected_date = None
# 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'
# 'Windows-10-10.0.19045-SP0'
selected_platform = None

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterRecursiveMultiSeries', 'ForecasterAutoregMultiSeries']")
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'")
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'")
    if selected_date:
        df = df[df['datetime'].dt.date == pd.to_datetime(selected_date).date()]
    if selected_platform:
        df = df[df['platform'] == selected_platform]
    if display_df:
        display(df)
    plot_benchmark_results(df.copy(), function_name, add_median=True, add_mean=True)

,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
0,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.290554,0.785076,2025-04-23 15:09:20.254703,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
12,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,11.181269,0.577565,2025-04-23 15:13:23.496449,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
24,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.332075,1.001781,2025-04-23 15:25:24.855452,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
36,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.542493,0.606813,2025-04-23 15:34:44.825479,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
56,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,10.987336,0.396632,2025-04-23 15:54:09.384976,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
68,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,11.761212,0.358871,2025-04-23 15:58:20.920219,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
80,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,11.800770,0.402881,2025-04-23 16:02:51.964754,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
92,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.650917,1.080875,2025-04-23 20:16:08.414338,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
105,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,12.449832,0.639767,2025-04-23 21:57:12.603493,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
118,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,13.398143,0.542395,2025-04-23 22:10:14.555069,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
1,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.730657,0.036684,2025-04-23 15:10:21.711978,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
13,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.008426,0.020794,2025-04-23 15:14:19.407835,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
25,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.624174,0.090886,2025-04-23 15:26:26.520388,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
37,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.603322,0.046577,2025-04-23 15:35:47.542332,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
57,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.998299,0.020942,2025-04-23 15:55:04.326742,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
69,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.760219,0.029486,2025-04-23 15:59:19.730993,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
81,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.708451,0.049639,2025-04-23 16:03:50.974931,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
93,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.778144,0.056622,2025-04-23 20:17:11.673544,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
106,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.678415,0.036097,2025-04-23 21:58:14.858324,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
119,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.350689,0.146020,2025-04-23 22:11:21.550881,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
2,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,10.828553,0.107749,2025-04-23 15:10:30.369368,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
14,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,10.099889,0.160311,2025-04-23 15:14:29.454622,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
26,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.003430,0.223845,2025-04-23 15:26:39.645552,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
38,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.890319,0.521044,2025-04-23 15:36:00.563508,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
58,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,10.926485,0.812523,2025-04-23 15:55:14.322641,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
70,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.199184,0.096970,2025-04-23 15:59:28.536409,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
82,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,11.031137,0.134091,2025-04-23 16:03:59.522090,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
94,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,13.922733,1.839202,2025-04-23 20:17:20.571857,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
107,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,12.949121,1.421564,2025-04-23 21:58:23.255455,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
120,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,12.067933,0.734881,2025-04-23 22:11:33.311522,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
3,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.375285,0.040809,2025-04-23 15:11:24.516068,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
15,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.517815,0.064871,2025-04-23 15:15:19.958170,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
27,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,2.004793,0.040390,2025-04-23 15:27:34.667213,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
39,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,2.146876,0.070924,2025-04-23 15:37:00.019066,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
59,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.568071,0.119750,2025-04-23 15:56:08.959300,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
71,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.411051,0.055234,2025-04-23 16:00:24.536982,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
83,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.385172,0.047952,2025-04-23 16:04:54.682880,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
95,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.406918,0.044804,2025-04-23 20:18:30.189994,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
108,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.521542,0.186202,2025-04-23 21:59:28.005422,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
121,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.744641,0.158125,2025-04-23 22:12:33.655628,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
4,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001370,0.000224,2025-04-23 15:11:43.049817,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
16,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001605,0.000359,2025-04-23 15:15:38.790638,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
28,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001624,0.000139,2025-04-23 15:27:57.642580,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
40,ForecasterAutoregMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.002253,0.000301,2025-04-23 15:37:24.534882,3.12.9,0.13.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
60,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.003732,0.000383,2025-04-23 15:56:31.510600,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
72,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001514,0.000252,2025-04-23 16:00:43.539847,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
84,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001539,0.000270,2025-04-23 16:05:13.551260,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
96,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001269,0.000178,2025-04-23 20:18:50.734185,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
109,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001208,0.000087,2025-04-23 21:59:49.656766,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64
122,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001409,0.000156,2025-04-23 22:12:55.033529,3.12.9,0.14.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-21-generic-x86_64-with-glibc2.39,x86_64,8,16.64


...

,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
154,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,12.627921,0.484849,2025-04-24 10:38:45.612438,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
169,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,10.287719,0.213035,2025-04-24 10:44:51.118971,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
184,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,10.403703,0.137579,2025-04-24 11:12:07.236935,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
199,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,12.782420,0.155627,2025-04-24 11:22:05.448858,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
215,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,10.698136,0.126667,2025-04-24 18:06:07.947284,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
234,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,9.375887,0.114813,2025-04-24 18:20:55.233667,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
253,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,13.709724,0.708862,2025-04-24 22:39:42.602392,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
272,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,18.186790,1.279906,2025-04-24 22:58:30.203036,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
362,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,15.446649,0.980125,2025-04-25 22:53:40.232185,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
207,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ace3e84de8c6ee0baf7abe80c790a1db,1.580161,0.075582,2025-04-24 18:02:30.636992,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
226,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ace3e84de8c6ee0baf7abe80c790a1db,1.376138,0.041489,2025-04-24 18:17:40.756467,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
245,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ace3e84de8c6ee0baf7abe80c790a1db,1.401489,0.035080,2025-04-24 22:34:58.197359,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
264,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ace3e84de8c6ee0baf7abe80c790a1db,2.225819,0.033479,2025-04-24 22:50:47.016292,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
353,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ace3e84de8c6ee0baf7abe80c790a1db,1.616509,0.038344,2025-04-25 22:48:20.386213,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
216,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,d72f22d955e2149418d02e9e4f8b9a45,15.007106,0.229013,2025-04-24 18:07:23.017057,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
235,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,d72f22d955e2149418d02e9e4f8b9a45,13.279067,0.246881,2025-04-24 18:22:01.650838,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
254,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,d72f22d955e2149418d02e9e4f8b9a45,17.794282,0.289894,2025-04-24 22:41:11.578988,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
273,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,d72f22d955e2149418d02e9e4f8b9a45,40.861020,1.097515,2025-04-24 23:01:54.513729,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
363,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,d72f22d955e2149418d02e9e4f8b9a45,20.219225,0.399932,2025-04-25 22:55:21.334890,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
217,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,4081578249b9beefbee5821991bb788b,23.776374,0.343084,2025-04-24 18:09:21.934592,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
236,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,4081578249b9beefbee5821991bb788b,25.542821,1.557417,2025-04-24 18:24:09.396025,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
255,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,4081578249b9beefbee5821991bb788b,28.694565,0.216067,2025-04-24 22:43:35.056748,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
274,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,4081578249b9beefbee5821991bb788b,50.676833,0.731038,2025-04-24 23:06:07.903988,3.12.9,0.15.1,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64
364,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,4081578249b9beefbee5821991bb788b,32.100457,0.211030,2025-04-25 22:58:01.842960,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
358,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_predict_interva...,076d0019387c7afe10d13a284909fd4a,NaN,NaN,2025-04-25 22:50:26.951238,3.12.9,0.16.0,2.2.5,2.2.3,1.6.1,4.6.0,Linux-6.11.0-24-generic-x86_64-with-glibc2.39,x86_64,8,16.64


# ForecasterRecursive

In [10]:
# Mock data for benchmarking
# ==========================================================
len_series = 2000
rng = np.random.default_rng(321)
y = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name='y'
    )
rng = np.random.default_rng(321)
exog = pd.DataFrame(index=y.index)
exog['day_of_week'] = exog.index.dayofweek
exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
exog['month'] = exog.index.month
exog_prediction = pd.DataFrame(
                    index=pd.date_range(
                        start=exog.index.max() + pd.Timedelta(hours=1),
                        periods=100,
                        freq='h'
                    )
                 ) 
exog_prediction['day_of_week'] = exog_prediction.index.dayofweek
exog_prediction['week_of_year'] = exog_prediction.index.isocalendar().week.astype(int)
exog_prediction['month'] = exog_prediction.index.month

In [11]:
# Benchmarking speed ForecasterRecursive
# ==============================================================================
regressor = LGBMRegressor(random_state=8520, verbose=-1)
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursive(
        regressor=regressor,
        lags=50,
        transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    )
else:
    forecaster = ForecasterAutoreg(
        regressor=regressor,
        lags=50,
        transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    )

def ForecasterRecursive_fit():
    forecaster.fit(y=y, exog=exog)

def ForecasterRecursive__create_train_X_y():
    if skforecast.__version__ >= '0.14.0':
        forecaster._create_train_X_y(y=y, exog=exog)
    else:
        forecaster.create_train_X_y(y=y, exog=exog)


def ForecasterRecursive_predict():
    forecaster.predict(steps=100, exog=exog_prediction)

def ForecasterRecursive_predict_interval_conformal():
    forecaster.predict_interval(
        steps=100,
        exog=exog_prediction,
        interval=[5, 95],
        method='conformal'
    )


def ForecasterRecursive__create_predict_inputs():
    _ = forecaster._create_predict_inputs(
            steps        = 100,
            exog         = exog_prediction,
            check_inputs = True
        )
    
def ForecasterRecursive_backtesting():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )

def ForecasterRecursive_backtesting_conformal():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                interval=[5, 95],
                interval_method='conformal',
                cv=cv,
                metric='mean_squared_error',

                show_progress=False
            )
    else:
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                interval=[5, 95],
                interval_method='conformal',
                metric='mean_squared_error',
                show_progress=False
            )


runner = BenchmarkRunner(repeat=30, output_dir="./")
_ = runner.benchmark(ForecasterRecursive__create_train_X_y, forecaster=forecaster)

runner = BenchmarkRunner(repeat=10, output_dir="./")
_ = runner.benchmark(ForecasterRecursive_fit, forecaster=forecaster)

runner = BenchmarkRunner(repeat=30, output_dir="./")
forecaster.fit(y=y, exog=exog, store_in_sample_residuals=True)
_ = runner.benchmark(ForecasterRecursive_predict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursive_predict_interval_conformal, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursive__create_predict_inputs, forecaster=forecaster)

runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursive_backtesting, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursive_backtesting_conformal, forecaster=forecaster)

Benchmarking function: ForecasterRecursive__create_train_X_y with skforecast version 0.16.0
Benchmarking function: ForecasterRecursive_fit with skforecast version 0.16.0
Benchmarking function: ForecasterRecursive_predict with skforecast version 0.16.0
Benchmarking function: ForecasterRecursive_predict_interval_conformal with skforecast version 0.16.0
Benchmarking function: ForecasterRecursive__create_predict_inputs with skforecast version 0.16.0
Benchmarking function: ForecasterRecursive_backtesting with skforecast version 0.16.0
Benchmarking function: ForecasterRecursive_backtesting_conformal with skforecast version 0.16.0


In [12]:
# Plot results
# ==============================================================================
display_df = False
selected_date = None
# 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'
# 'Windows-10-10.0.19045-SP0'
selected_platform = None

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterRecursive', 'ForecasterAutoreg']")
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'")
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'")
    if selected_date:
        df = df[df['datetime'].dt.date == pd.to_datetime(selected_date).date()]
    if selected_platform:
        df = df[df['platform'] == selected_platform]
    if display_df:
        display(df)
    plot_benchmark_results(df.copy(), function_name, add_median=True, add_mean=True)

# Summary of historical results

In [13]:
selected_date = None
# 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'
# 'Windows-10-10.0.19045-SP0'
selected_platform = None

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark_all['function_name'] = results_benchmark_all['function_name'].str.split('_', n=1).str[1]
if selected_date:
    results_benchmark_all = results_benchmark_all.query("datetime.dt.date == @selected_date")
if selected_platform:
    results_benchmark_all = results_benchmark_all.query("platform == @selected_platform")
results_benchmark_all = results_benchmark_all.groupby(['forecaster_name', 'skforecast_version', 'function_name'])['run_time_avg'].agg('median').reset_index()
results_benchmark_all = results_benchmark_all.sort_values(by=['function_name'])

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterRecursiveMultiSeries', 'ForecasterAutoregMultiSeries']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='MultiSeries Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterRecursive', 'ForecasterAutoreg']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='Single-Series Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()
